Install the necessary libraries - Tested with Python 3.10

In [ ]:
!pip install numpy==1.24.2 torch==2.0.0 anytree transformers==4.27.4 safetensors sentencepiece

Import the necessary libraries

In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
from lrml import *

/Users/stefanfuchs/Repos/lrml-editor/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load the model from Huggingface

In [2]:
model_path = 'sffc348/t5-base-lrml-autocomplete'
model = T5ForConditionalGeneration.from_pretrained('sffc348/t5-base-lrml-autocomplete', use_safetensors=True)
model.eval()

/Users/stefanfuchs/Repos/lrml-editor/.conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [3]:
tokenizer_name = 't5-base'

def load_tokenizer():
    tokenizer = T5Tokenizer.from_pretrained(tokenizer_name)
    tokenizer.add_tokens(['<sep>'], special_tokens=True)
    tokenizer.sep_token = '<sep>'
    tokenizer.sep_token_id = tokenizer.convert_tokens_to_ids(
        tokenizer.sep_token)
    return tokenizer

tokenizer = load_tokenizer()

/Users/stefanfuchs/Repos/lrml-editor/.conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [4]:
# Preprocessing of text
def normalise_text(text):
    text = text.strip()
    if text and text[-1] != '.':
        text += '.'
    return text

# Generate a prediction based on the input text and the LRML
def predict(text, lrml):
    num_beams = 5
    num_return_sequences = 5
    no_repeat_ngram_size = 8
    max_length = 256
    early_stopping = True
    print(text, lrml)
    if lrml.strip() != '':
        lrml = '<sep>' + lrml
    else:
        lrml = ''
    tokens = tokenizer('translate English to LegalRuleML: ' +
                            normalise_text(text) + lrml, return_tensors='pt')
    with torch.no_grad():
        generation = model.generate(inputs=tokens.input_ids, max_length=max_length, num_beams=num_beams,
                                            num_return_sequences=num_return_sequences, early_stopping=early_stopping,
                                            no_repeat_ngram_size=no_repeat_ngram_size)

    return [post_process(i) for i in tokenizer.batch_decode(generation, skip_special_tokens=True)]

# Functions for postprocessing
def post_process(lrml):
    lrml = lrml.strip()
    # lrml = lrml[lrml.find('if('):]
    lrml = lrml.replace('[', '(').replace(']', ')').replace(
        '{', '(').replace('}', ')')
    lrml = lrml.replace(').', ')')
    lrml = fix_then(lrml, ' ')
    lrml = revert_tree_based_spacing(lrml)
    lrml = add_space_after_comma(lrml)

    return lrml

def clean_pred(lrml):
    prefix = ''
    lrml = lrml.replace(', ', ',')

    lrml = reverse_loop(lrml, prefix=prefix)
    lrml = reverse_resolve_expressions(lrml, fix_errors=True, prefix=prefix)
    lrml = reverse_combine_rel_and_var(lrml, prefix=prefix)
    lrml = reverse_move_and_or_to_data_node(lrml)
    lrml = reverse_units(lrml, prefix=prefix)
    return lrml

Do the actual prediction, the LRML input decides the scope of the prediction

In [5]:
text = 'The floor waste shall have a minimum diameter of 40 mm.'
# lrml = ''
lrml = 'if('
lrml = 'if(exist(floorWaste)), then(obligation('
predictions = predict(text, lrml)
predictions

The floor waste shall have a minimum diameter of 40 mm. if(exist(floorWaste)), then(obligation(


['greaterThanEqual(floorWaste.diameter, 40 mm)',
 'lessThanEqual(floorWaste.diameter, 40 mm)',
 'equal(floorWaste.diameter, 40 mm)',
 'exceed(floorWaste.diameter, 40 mm)',
 'greaterThanEqual(floorWaste.diameter, 40 mm)']

Post-process the output

In [6]:
clean_pred(predictions[0])

'expression(function(greaterThanEqual),atom(relation(diameter),variable(floorWaste)),data(baseunit(prefix(milli),kind(metre)),value(40.0)))'

In [7]:
test_string = clean_pred(lrml)
test_string

'if(expression(function(exist),atom(variable(floorWaste)))),then(obligation)'

In [11]:
from lrml_long_to_short import *

def get_short_lrml(lrml):
    lrml = resolve_units(lrml)
    lrml = move_and_or_to_data_node(lrml)
    lrml = combine_rel_and_var(lrml)
    lrml = resolve_expressions(lrml)
    lrml = resolve_loop(lrml)
    return lrml

get_short_lrml(test_string)

'if(exist(floorWaste)),then(obligation)'

In [5]:
from lrml_long_to_short import *

test_string = 'if(expr(atom('

get_short_lrml(test_string)

Error:  ()
Error:  Node('/root/if/expr', node_id=1) (Node('/root/if/expr/atom', node_id=2),) () if(expr(atom))
Node('/root/if/expr', node_id=1) (Node('/root/if/expr/atom', node_id=2),)


'if(expr(atom))'

In [10]:
from lrml_long_to_short import *

test_string = 'if(expr(atom(test))'

get_short_lrml(test_string)

Error:  (Node('/root/if/expr/atom/test', node_id=3),)
Error:  Node('/root/if/expr', node_id=1) (Node('/root/if/expr/atom', node_id=2),) () if(expr(atom(test)))
Node('/root/if/expr', node_id=1) (Node('/root/if/expr/atom', node_id=2),)


'if(expr(atom(test)))'

In [13]:
from lrml_long_to_short import *

test_string = 'if(expr(fun(greater),atom(test))'

get_short_lrml(test_string)

Error:  (Node('/root/if/expr/atom/test', node_id=5),)


'if(greater(test))'